In [1]:
# from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate
from sklearn.experimental import enable_hist_gradient_boosting  # noqa: F401
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import roc_auc_score, average_precision_score, classification_report
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate
from joblib import dump
import pandas as pd
import json, os, re, sys

C:\Users\cbreu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\experimental\enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


In [11]:
# Read in the data

df = pd.read_parquet('C:/test/data/9_clean_in_py/training.pycleaned.parquet')

In [12]:
# Copy to training dataset

X = df.copy()

In [14]:
# Predictor column (0/1 for benign/pathogenic)

y = X["label"]

In [15]:
# Drop unneeded columns (target column is 'label' with "benign"/0, "pathogenic"/1)

X = X.drop(["chrom","pos","ref","alt", "label"], axis =1 )

In [18]:
# All remaining columns should be numeric. 
# Double check to ensure numeric dtype

for c in X.columns:
    X[c] = pd.to_numeric(X[c], errors="coerce")

y = pd.to_numeric(y, errors="coerce")

In [19]:
# Print a rough summary of target column

benign = sum(y==0)
pathogenic = sum(y==1)
total = benign + pathogenic
pct = int(100*(benign/total))

print(f'Number of predictors: \n Benign: {benign} \n Pathogenic: {pathogenic} \n Percent Pathogenic: {pct}')

Number of predictors: 
 Benign: 1263888 
 Pathogenic: 460877 
 Percent Pathogenic: 73


In [20]:
# Set random number seed

randomSeed = 99

In [22]:
# Train/test split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=randomSeed, stratify=y)

In [23]:
# Pipeline settings

# Pipeline: median imputer -> HistGradientBoosting

pipe = Pipeline(steps=[
    ("impute", SimpleImputer(strategy="median")),
    ("clf", HistGradientBoostingClassifier(
        learning_rate=0.05,
        max_depth=None,      # allow tree growth with L2 regularization
        max_leaf_nodes=31,
        l2_regularization=0.0,
        random_state=randomSeed
    ))
])

In [ ]:
# Test the effectiveness of the training data + model + params using 5-fold cross validation
    # This creates a model 5x on 5 different train/test splits
    # This does not produce an output model. It only establishes the likelihood the settings will produce a good model.

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=randomSeed)

# Set scoring method
    # ROC AUC - area under the curve - higher = better
    # Average precision - good for cases where the positive value is rare (not the case here, but ok)
scoring = {"roc_auc":"roc_auc", "pr_auc":"average_precision"}

# Test by 5x cross validation
cv_res = cross_validate(pipe, X_train, y_train, cv=cv, scoring=scoring, n_jobs=-1, return_train_score=False)

In [18]:
# Report CV test results

print(f"[cv] ROC-AUC mean={cv_res['test_roc_auc'].mean():.3f} ± {cv_res['test_roc_auc'].std():.3f}")
print(f"[cv] PR-AUC  mean={cv_res['test_pr_auc'].mean():.3f} ± {cv_res['test_pr_auc'].std():.3f}")

[cv] ROC-AUC mean=0.960 ± 0.000
[cv] PR-AUC  mean=0.933 ± 0.000


In [19]:
# Fit on full train; evaluate on held-out test

pipe.fit(X_train, y_train)
proba = pipe.predict_proba(X_test)[:,1]
preds = (proba >= 0.5).astype(int)

In [20]:
# Check out Proba - probability that each predictor is the positive class (i.e. a "1" == "pathogenic")

proba

array([0.02964491, 0.02347764, 0.03919975, ..., 0.16676153, 0.02385342,
       0.04745789])

In [21]:
# Assess model

test_roc = roc_auc_score(y_test, proba)
test_pr  = average_precision_score(y_test, proba)

print(f"[test] ROC-AUC={test_roc:.3f}  PR-AUC={test_pr:.3f}")
print("[test] classification report:")
print(classification_report(y_test, preds, digits=3))

[test] ROC-AUC=0.960  PR-AUC=0.933
[test] classification report:
              precision    recall  f1-score   support

           0      0.931     0.974     0.952    252778
           1      0.918     0.802     0.856     92175

    accuracy                          0.928    344953
   macro avg      0.924     0.888     0.904    344953
weighted avg      0.927     0.928     0.926    344953



In [36]:
# Save the model
pipelineName = "model1.joblib"
dump(pipe, pipelineName)

['model1.joblib']

In [44]:
# Record model features

with open("modelFeatures.txt", "w") as out:
    out.write("Features:\n")
    for f in X.columns:
        out.write(f + "\n")

In [48]:
# Save model scores

with open("modelScores.json", "w") as f:
    json.dump({
        "cv_roc_auc_mean": float(cv_res['test_roc_auc'].mean()),
        "cv_pr_auc_mean": float(cv_res['test_pr_auc'].mean()),
        "test_roc_auc": float(test_roc),
        "test_pr_auc": float(test_pr)
    }, f, indent=2)

print(f"[done] saved model → {'variant_model.joblib'}")
print(f"[done] saved features → {'features.json'}")
print(f"[done] saved metrics → {'metrics.json'}")

[done] saved model → variant_model.joblib
[done] saved features → features.json
[done] saved metrics → metrics.json
